# _qhe-library DOCUMENTATION<br>
Updated 2021-07-01

This is a documentation for my FQHE library<br>
A version of this library is available on both of Prof Yang's workstation.<br>
To access them, add the following at the begining of your Python script:

Alternatively, you can copy relevant modules to your own subfolder and import from there.

Although I am constantly updating this library, I (try to)  make every update backward-compatible, so all of the things in these documents should still work for future updates.

In each of the module explained below, I also indicate dependency on any of the other modules in the library. This means that if you want to use a module x.py that is also dependent on y.py and z.py, you must put x.py, y.py, and z.py in your current directory. (If you add the import sys.... as above, you don't have to worry about this.)

Of course, I assume you already have scipy, numpy, and sympy.

### I. The FQH_states module
> Dependency: misc.py

The first useful module is **FQH_states**. The most important feature in this module is the class **fqh_states**: 

In [ ]:
from FQH_states import *


x = fqh_state()
print(x.basis)
print(x.coef)

When there is no argument (such as in the above example), and empty state will be initialized. An fqh_state variable has two main attributes, **.basis**, which stores the basis states, and **.coef**, which stores the corresponding coefficient.

A state can be initialized by either of the following two methods

###### METHOD 1: Specify the basis and corresponding coefficient

In [ ]:
basis = ["1001", "0110"]
coef  = [1,-3]

psi = fqh_state((basis, coef)) # Note that the argument is a 2-element tuple
print(psi.basis)
print(psi.coef)

print("----")
# The basis can either be in binary format (as above) or decimal:
basis_decimal = [9,6]
psi_2 = fqh_state((basis_decimal,coef))
print(psi_2.basis)
print(psi_2.coef)


###### METHOD 2: Specify the text file that contains the wavefunction

The text file must be in the standard format (explained below)

In [ ]:
psi_3 = fqh_state("sample_state_bin")
print(psi_3.basis)
print(psi_3.coef)
print(psi_3.format)

print("---")
# It will automatically detect the basis format in the file.
# Above shows an example of a wavefunction stored in binary format.
# An example of a wavefunction stored in decimal is as follows
psi_4 = fqh_state("sample_state_dec")
print(psi_4.basis)
print(psi_4.coef)
print(psi_4.format)


**.format_convert()** is used to convert between binary and decimal. The code will detect the current format the vector is in, and convert it to the other.

When converting from decimal to binary, an additional input of *N_orb* is required

**NOTE THAT in this update, this conversion only works for fermionic states. For bosonic state, all other features of this class is still useable (in particular the vector space operations) but not the conversion**

In [ ]:
psi_3.format_convert()
print(psi_3.basis)
print(psi_3.coef)
print(psi_3.format)

print("---")
# Converting from decimalto binary requires an additional input of N_orbital
psi_4.format_convert(No=4)
print(psi_4.basis)
print(psi_4.coef)
print(psi_4.format)

# If N_orb is not specified, the program will prompt user to input it manually

When converting wavefunction format (either binary to decimal or decimal to binary), one can also choose to *invert* all the basis (i.e. reading all the binary roots backward):

In [ ]:
from FQH_states import *
psi = fqh_state((["01001","00110"], [1,-3]))

print(psi.basis)

psi.format_convert(invert = True) # convert to decimal with inversion
psi.format_convert(No=5)         # convert back to binary to compare
print(psi.basis)

This feature exist because according to Prof Yang's convention, **left-most digit is the north pole**, while I wrote all my code with the convention that the **left-most digit is the south pole**. 

("south pole" means Lz = -S on the sphere or m=0 on the disk)

For example, "01001" is 2^1+2^4 = 18 in my convention, but is 2^0+2^3 = 9 in Prof Yang's convention. *Our conventions agree on the decimal representation, but disagree on the binary convention*.

Inversion is needed if one wants to incorporate both Prof Yang's and mine routines in a single task.

**.printwf()** is used to display the wavefunction in *traditional format*. This format starts with the dimension of the state, followed by pairs of basis vectors (in either binary or decimal format) and the corresponding coefficients.

If no argument is supplied, **.printwf()** will print the state to screen. A string variable can be supplied to save the state to a text file with that name.

In [ ]:
basis = ["1001", "0110"]
coef  = [1,-3]

psi = fqh_state((basis, coef))

psi.printwf()

In [ ]:
psi.printwf("test.txt") # You can check the current directory for this new file

**fqh_state** variables come with inbuilt standard operations on a vector space, namely:

##### Scalar multiplication:

In [ ]:
a = 2*psi
a.printwf()

###### Addition:

In [ ]:
b = fqh_state((["1001001","0110001"], [0.2,0.3]))
print("b =")
b.printwf()

c = fqh_state((["1001001", "1000110"], [0.1,0.4]))
print("c = ")
c.printwf()

d = b + c
print("b+c =")
d.printwf()

###### Inner product:

In [ ]:
print("<b|c> =")
print(b.overlap(c))

There are currently two normalization schemes available:

1. **.normalize()** for the usual normalization
2. **.sphere_normalize()** to convert a pure jack polynomial to the wavefunction on the sphere

In [ ]:
psi = fqh_state((["1001","0110"], [1,-3]))
psi.normalize()
psi.printwf()
print("norm = ")
print(psi.norm())

In [ ]:
psi = fqh_state((["1001","0110"], [1,-3]))
psi.sphere_normalize()
psi.printwf()
print("norm = ")
print(psi.norm())

It will also be useful to get what I call the "index" format of the basis vectors. The index format of each basis vector is a list of orbital indices that are occupied by an electrons. 0 indexes the left-most orbital and *N_orb-1* indices the right-most orbital.

One can obtain this index format by **.get_basis_index()**. As always, if the state is in decimal format, *N_orb* must be supplied as an argument.

In [ ]:
psi = fqh_state((["1001","0110"], [1,-3]))
bas = psi.get_basis_index()
print(bas)

There is another variable class, **fqh_state_boson**, that has almost all of the above features, but is used for bosonic state. The difference is that in "binary" representation, bosonic state can have digits greater than 1 because Pauli exclusion princle doesn't hold.

For this class, every function above holds except for **.normalize_sphere()** and basis format conversion from decimal to binary (but **.format_convert()** works fine for binary-to-decimal.

### II. The task1 module

> Dependency: misc.py, angular_momentum.py

This was the first module I have ever written, hence the naming is quite silly. However it does contain some functions that are still useful (and I have updated it eversince to optimize some routines).

In [ ]:
import task1

Below are some important functions in this module

**task1.findBasis_brute(No,Ne,return_index=False, Lz=0, bosonic=False)** is a function that returns all basis states of a particular Hilbert space (no truncation). It takes the following variables:

1. Ne: Number of particles
2. No: Number of orbitals
3. return_index: set to True to return the index format (explained above). If set to False, it will return in an obsolete format that I no longer use, so don't bother. Just always remember to set this to True
4. Lz: The L_z sector, default = 0
5. bosonic: set to True if you want the bosonic states (more than one particle can occupy an orbital)

**task1.LplusLminus_2(basis_index, Ne, No)** returns the matrix L^+L^- FOR FERMIONIC STATE ONLY (for bosonic equivalent see angular_momentum module below). There are multiple version of this routine in the module (hence the suffix "_2"), but this is the fastest. This function takes the following variables

1. basis_index: list of all basis vectors in index format, i.e. the output of findBasis_brute() above
2. Ne: Number of electrons
3. No: Number of orbitals

**task1.get_highest_weight(Ne,No,Lz, bosonic=False, basisformat="index")** returns a tuple (b,z) where b is a list of basis and z is a dim-by-N array where dim is the dimension of the full Hilbert space in the given L_z sector and N is the number of highest-weight state in that sector. It takes the following variables:

1. Ne: Number of particles
2. No: Number of orbitals
3. Lz: The L_z sector (no default value)
4. bosonic: set to True to find the bosonic state
5. basisformat: set to either "index", "dec" for decimal, or "bin" for binary. Either of this three options works as they should.

NOTE THAT if bosonic = True, it will return the highest-weight states whereas if bosonic = False,it will return the LOWEST-WEIGHT states. Lowest weight can be converted to highest-weight by inverting the vector. 

Below are two examples.

In [1]:
import task1
import FQH_states as FQH

# Find the __fermionic__ highest weight states:
(b,z) = task1.get_highest_weight(3, 8, -1.5, basisformat="bin")

# There should be one lowest weight state that is the Laughlin one-quasihole state
print(f"# of lowest-weight state: {len(z)}")
print()

state = FQH.fqh_state((b,z[0]))
print("lowest-weight state = ")
state.printwf()

# Convert to highest-weight by invert():
state.invert()
print("highest-weight state =")
state.printwf()

# of lowest-weight state: 1

lowest-weight state = 
6
10100001
-0.0
10010010
0.3162277660168378
10001100
-0.5477225575051659
01100010
-0.4629100498862757
01010100
0.41403933560541273
00111000
-0.4629100498862762

highest-weight state =
6
10000101
-0.0
01001001
0.3162277660168378
00110001
-0.5477225575051659
01000110
-0.4629100498862757
00101010
0.41403933560541273
00011100
-0.4629100498862762



In [3]:
import task1
import FQH_states as FQH

# Find the __bosonic__ highest-weight state:
(b,z) = task1.get_highest_weight(4, 6, 0, bosonic=True,basisformat="bin")
print(len(z))

state = FQH.fqh_state_boson((b,z[0]))
print("highest-weight state = ")
state.printwf()

state.format_convert()

1
highest-weight state = 
12
2 0 0 0 0 2
0.42470599286468785
1 1 0 0 1 1
-0.42470599286468774
1 0 1 1 0 1
0.0849411985729374
1 0 1 0 2 0
0.3038948705590346
1 0 0 2 1 0
-0.16116459280507608
0 2 0 1 0 1
0.3038948705590348
0 2 0 0 2 0
0.1528941574312877
0 1 2 0 0 1
-0.16116459280507614
0 1 1 1 1 0
-0.322776554577163
0 1 0 3 0 0
0.3530939318018999
0 0 3 0 1 0
0.35309393180189996
0 0 2 2 0 0
-0.13590591771670024



### III. The task2 module

This module contains the basic functions for the LEC formalism.


### IV. The angular_momentum module
> Dependency: misc.py

I wrote this before writing the **FQH_states** module therefore for all of the functions in this module, one must input the basis and coefficient as separate variables (rather than as one variable of the fqh_state class).

This module contains four main functions: **Lplus, Lminus, Lplus_boson, Lminus_boson**. The ones without _boson are fermionic operators. The input arguments are the same for all four of them so I just use Lplus as an example:

**angular_momentum.Lplus(basis_list, coef_list, No, debug=False, normalize=True)**:

1. basis_list: a list of all basis vector **in decimal format**
2. coef_list: the list of correspondoing coefficients
3. No: number of orbitals
4. debug: set to True to print out intermediate output for debugging purpose. Useful if you would like to modify the source code.
5. normalize = True: normalize the resultant state.

Output: b, c where b is the list of basis states and c is the list of corresponding coefficient in the resultant state.

This module also contains functions to construct the L^+L^- matrix for both bosons and fermions. LplusLminus (for fermionic) here isn't as efficient as **task1.LplusLminus_2**, so don't bother. The bosonic version is:

**angular_momentum.LminusLplus_boson(basis_list_index, No)**:

1. basis_list_index: a list of all basis vectors **in index format** (same requirement as task1.LplusLminus_2)
2. No: Number of orbitals

For bosonic, this is L^-L^+ instead of L^+L^- because it's the "particle-hole conjugate" of the fermionic L^+L^-.


### V. The misc module
> Dependency: none

This contains a lot of foundational functions used in other modules in the library. It also contains a lot of standalone functions that used to be directly useable (such as reading and saving wavefunction files and converting file format). However most of these have been rendered obsolete after completion of the FQH_states module. For now I won't document it here.